In [2]:
import os
import datetime, json
import requests
from azure.ai.anomalydetector import AnomalyDetectorClient
from azure.ai.anomalydetector.models import DetectRequest, TimeSeriesPoint, TimeGranularity, \
    AnomalyDetectorError
from azure.core.credentials import AzureKeyCredential
import pandas as pd
from pymongo import MongoClient
from dateutil import parser


In [30]:
SUBSCRIPTION_KEY = ''
ANOMALY_DETECTOR_ENDPOINT = ''
TIME_SERIES_DATA_PATH = os.path.join("./AnomalyDetector/example-data/", "request-data.csv")

In [31]:
client = MongoClient(host="localhost", port=27017)

In [32]:
db = client['v-dat']
sensor = db['sensor']

In [33]:
j = {'granularity': 'daily'}
t = []
for i in sensor.find():
    t.append({'timestamp': i['timestamp'],'value': i['value']})
j['series'] = t

In [34]:
def json_default(value): 
    if isinstance(value, datetime.date): 
        return value.strftime('%Y-%m-%d') 
    raise TypeError('not JSON serializable')


def detect(endpoint, apikey, request_data):
    headers = {'Content-Type': 'application/json', 'Ocp-Apim-Subscription-Key': apikey}
    response = requests.post(endpoint, data=json.dumps(request_data, default=json_default), headers=headers)
    if response.status_code == 200:
        return json.loads(response.content.decode("utf-8"))
    else:
        print(response.status_code)
        raise Exception(response.text)

In [104]:
# sample_data = json.load(open(TIME_SERIES_DATA_PATH))
result = detect(ANOMALY_DETECTOR_ENDPOINT, SUBSCRIPTION_KEY, sendBvp)
result = {'expectedValues': result['expectedValues'], 'isAnomaly': result['isAnomaly'], 'isNegativeAnomaly': result['isNegativeAnomaly'],
          'isPositiveAnomaly': result['isPositiveAnomaly'], 'upperMargins': result['upperMargins'], 'lowerMargins': result['lowerMargins'],
          'timestamp': [x['timestamp'] for x in p['series']], 
          'value': [x['value'] for x in p['series']]}


KeyError: 'expectedValues'

In [35]:
def detect_anomaly(sensitivity,data):
    points = data['series']
    result = {'expectedValues': [None]*len(points), 'isAnomaly': [False]*len(points)}
    anom_count = 0
    for i in range(0, len(points)+1):
        single_sample_data = {}
        single_sample_data['series'] = points
        single_sample_data['granularity'] = 'secondly'
        single_sample_data['maxAnomalyRatio'] = 0.25
        single_sample_data['sensitivity'] = sensitivity
        single_point = detect(ANOMALY_DETECTOR_ENDPOINT, SUBSCRIPTION_KEY, single_sample_data)
        if single_point['isAnomaly'] == True:
            anom_count = anom_count + 1
        print(single_point)
        result['expectedValues'][i-1] = single_point['expectedValue']
#         result['upperMargins'][i-1] = single_point['upperMargin']
#         result['lowerMargins'][i-1] = single_point['lowerMargin']
#         result['isNegativeAnomaly'][i-1] = single_point['isNegativeAnomaly']
#         result['isPositiveAnomaly'][i-1] = single_point['isPositiveAnomaly']
        result['isAnomaly'][i-1] = single_point['isAnomaly']
#         result['timestamp'][i-1] = data['series']['timstamp'][i] 
#         result['value'][i-1] = 
    
    return result

In [36]:
import time
data = pd.read_csv('./00.csv')
data = data.drop(index=0)
data = data.fillna(0)
data = data.reset_index(drop=True)
data['time'] = [i.replace(" ", "") for i in data[' Time']]
data['unix'] = [time.mktime(datetime.datetime.strptime(str(data['Date'][i])+data['time'][i],'%Y%m%d%H:%M:%S').timetuple())
                for i in range(0, len(data))]
datatime = data[['Date', 'time']]
vrDate = datatime['Date'][0]
vrTime = []
for i in sorted(datatime['time']):
    t = datetime.datetime.strptime(str(vrDate)+ ' ' + i, "%Y%m%d %H:%M:%S")
    vrTime.append(t.isoformat()+'T')

data['vrTime'] = vrTime
print(data['vrTime'])
data = data.drop(' Time', axis=1)

incol = []
for i in data['unix']:
    incol.append(i - data['unix'][0])
data['idx'] = incol
# end = incol[-1]


vrTime = data[['idx','vrTime']]
hmd = data[['idx','unix', ' Left_pos.x', ' Left_pos.y', ' Left_pos.z',
               ' Left_rot.x', ' Left_rot.y', ' Left_rot.z']]
e4Eda = data[[' EDA']]
e4Bvp = data[[' BVP']]
e4Tmp = data[[' TMP']]
# e4Ibi = data[[' IBI']]
eyeTracker = data[['idx','unix', ' combined_x', ' combined_y', ' combined_z', ' r_bit', ' r_open',
               ' r_dir_x', ' r_dir_y', ' r_dir_z', ' r_gaze_origin_x',
               ' r_gaze_origin_y', ' r_gaze_origin_z', ' r_pupil', ' r_pupil_pos',
               ' r_pupil_pos_x', ' r_pupil_pos_y', ' r_frown', ' r_squeeze',
               ' r_widel_bit', ' l_open', ' l_dir_x', ' l_dir_y', ' l_dir_z',
               ' l_gaze_origin_x', ' l_gaze_origin_y', ' l_gaze_origin_z', ' l_pupil',
               ' l_pupil_pos', ' l_pupil_pos_x', ' l_pupil_pos_y', ' l_frown',
               ' l_squeeze', ' l_wide']]

e4Eda = e4Eda[e4Eda[' EDA'] != ' default'] 
e4Bvp = e4Bvp[e4Bvp[' BVP'] != ' ']
e4Tmp = e4Tmp[e4Tmp[' TMP'] != ' ']  
e4Tmp = e4Tmp[e4Tmp[' TMP'] != ' R device_subscribe gsr O']
e4Tmp = e4Tmp[e4Tmp[' TMP'] != ' R device_subscribe bvp O']
e4Tmp = e4Tmp[e4Tmp[' TMP'] != ' R device_subscribe tmp O']
# e4Ibi = e4Ibi[e4Ibi[' IBI'] != ' ']

e4Eda = e4Eda.reset_index(drop=True)
e4Bvp = e4Bvp.reset_index(drop=True)
e4Tmp = e4Tmp.reset_index(drop=True)
# e4Ibi = e4Ibi.reset_index(drop=True)

e4Bvp = pd.DataFrame(data= [i[8:].split() for i in e4Bvp[' BVP']], columns=['unix','value'])
e4Eda = pd.DataFrame(data= [i[8:].split() for i in e4Eda[' EDA']], columns=['unix','value'])
e4Tmp = pd.DataFrame(data= [i[16:].split() for i in e4Tmp[' TMP']], columns=['unix','value'])        
# e4Ibi = pd.DataFrame(data= [i[8:].split() for i in e4Ibi[' IBI']], columns=['unix','value'])        
# e4Hr = e4Ibi.copy()
# a = np.full(len(e4Hr['value']), 60)
# e4Hr['value'] = a / np.array(e4Hr['value'], dtype=float)


0       2021-06-08T13:39:08T
1       2021-06-08T13:39:08T
2       2021-06-08T13:39:08T
3       2021-06-08T13:39:08T
4       2021-06-08T13:39:08T
                ...         
3762    2021-06-08T13:39:54T
3763    2021-06-08T13:39:54T
3764    2021-06-08T13:39:54T
3765    2021-06-08T13:39:54T
3766    2021-06-08T13:39:54T
Name: vrTime, Length: 3767, dtype: object


In [37]:
import re 
def processE4(data):
    start = int(vrTime['vrTime'][0][-2:-1])
    end = int(vrTime['vrTime'][len(vrTime)-1][-2:-1])

    
    unix = sorted(list(set([float(i) for i in data['unix']])))
    value = []
    iso = []    
    for i in unix:
        temp = [float(i) for i in data[data['unix'] == str(i)]['value']]
        value.append(round(sum(temp) / len(temp),4))
        t = float(i)
        iso.append(datetime.datetime.fromtimestamp(t).isoformat())
                
    pData = pd.DataFrame()
    pData['unix'] = unix
    pData['iso'] = iso
    pData['value'] = value
    
    pattern = re.compile(r':\d\d')
    sec = []
    for i in pData['iso']:
        sec.append(re.findall(pattern,i)[1][1:]) 
    
    pData['sec'] = sec

    mValue = []
    for i in set(pData['sec']):
        mValue.append(pData[pData['sec'] == i]['value'].mean())
        
    requestData = pd.DataFrame()
    requestData['timestamp'] = sorted(list(set([i[:19] for i in pData['iso']])))
    requestData['value'] = mValue
    
    sensorStart = int(requestData['timestamp'][0][-2:])
    sensorEnd = int(requestData['timestamp'][len(requestData)-1][-2:-1])
    
    temp = pd.DataFrame(columns=['timestamp', 'value'])

    if sensorEnd > end:
        requestData = requestData[:-(sensorEnd - end)]
    if sensorEnd <= end:
        t = []
        v = []
        for i in range(1, end - sensorEnd +1):
            t.append(datetime.datetime.strptime(requestData['timestamp'][len(requestData)-1], "%Y-%m-%dT%H:%M:%S") + datetime.timedelta(seconds=i))
            v.append(0)
        temp['timestamp'] = t
        temp['value'] = v
        requestData = requestData.append(temp).reset_index(drop=True)
    
        
    temp = pd.DataFrame(columns=['timestamp', 'value'])
    t = []
    v = []
    for i in range(0, sensorStart - start):
        t.append(datetime.datetime.strptime(vrTime['vrTime'][0], "%Y-%m-%dT%H:%M:%S") + datetime.timedelta(seconds=i))
        v.append(0)
    temp['timestamp'] = t
    temp['value'] = v
    
    requestData = temp.append(requestData).reset_index(drop=True)
    
    pData = pData[['unix', 'iso', 'sec', 'value']]

    return pData, requestData

def processAnother(data):
    idx = vrTime['idx']
    pData = pd.DataFrame(columns=[data.columns])
    timeList = []
    sec = []
    for i in list(set(idx)):
        t_data = data[data['idx'] == i]
        sec.append(list(vrTime[data['idx'] == i]['vrTime'])[0][-2:])
        timeList.append(list(vrTime[data['idx'] == i]['vrTime'])[0])
        pData.loc[len(pData)] = list(t_data.mean())
    pData['timestamp'] = timeList
    pData['sec'] = sec    
    
    return pData

def makeRequestHmd(data):
    
    requestPosX = pd.DataFrame()
    requestPosY = pd.DataFrame()
    requestPosZ = pd.DataFrame()
    
    requestPosX['timestamp'] = data['timestamp']
    requestPosX['value'] =  data[' Left_pos.x']
    
    requestPosY['timestamp'] = data['timestamp']
    requestPosY['value'] =  data[' Left_pos.y']
    
    requestPosZ['timestamp'] = data['timestamp']
    requestPosZ['value'] =  data[' Left_pos.z']
    
    return requestPosX, requestPosY, requestPosZ

def makeRequestEyetracker(data):
    requestCombinedX = pd.DataFrame()
    requestCombinedY = pd.DataFrame()
    requestCombinedZ = pd.DataFrame()
    
    requestCombinedX['timestamp'] = data['timestamp']
    requestCombinedX['value'] = data[' combined_x']

    requestCombinedY['timestamp'] = data['timestamp']
    requestCombinedY['value'] = data[' combined_y']

    requestCombinedZ['timestamp'] = data['timestamp']
    requestCombinedZ['value'] = data[' combined_z']
    
    return requestCombinedX, requestCombinedY, requestCombinedZ

def makeJson(data):
    p = {'granularity': 'secondly'}
    t = []
    for i, v in enumerate(data['timestamp']):
        t.append({'timestamp': str(v),'value': str(data['value'][i])})
    p['series'] = t
    
    return p


def getResult(data):
    result = detect(ANOMALY_DETECTOR_ENDPOINT, SUBSCRIPTION_KEY, data)
    print(result)
    result = {'expectedValues': result['expectedValues'], 'isAnomaly': result['isAnomaly'], 'isNegativeAnomaly': result['isNegativeAnomaly'],
          'isPositiveAnomaly': result['isPositiveAnomaly'], 'upperMargins': result['upperMargins'], 'lowerMargins': result['lowerMargins'],
          'timestamp': [x['timestamp'] for x in data['series']], 
          'value': [x['value'] for x in data['series']]}
    
    return result
 


In [38]:
pHmd = processAnother(hmd)
pHmd = pHmd[['idx','unix','timestamp', 'sec', ' Left_pos.x', ' Left_pos.y', ' Left_pos.z', 
             ' Left_rot.x', ' Left_rot.y',' Left_rot.z']]


pEyetracker = processAnother(eyeTracker)
pEyetracker = pEyetracker[['idx', 'unix','timestamp', 'sec', ' combined_x', ' combined_y', ' combined_z',
                           ' r_bit', ' r_open', ' r_dir_x', ' r_dir_y', ' r_dir_z', ' r_gaze_origin_x', 
                           ' r_gaze_origin_y',' r_gaze_origin_z',' r_pupil',' r_pupil_pos',' r_pupil_pos_x',
                           ' r_pupil_pos_y', ' r_frown', ' r_squeeze', ' r_widel_bit', ' l_open',
                           ' l_dir_x', ' l_dir_y', ' l_dir_z', ' l_gaze_origin_x', ' l_gaze_origin_y',
                           ' l_gaze_origin_z', ' l_pupil', ' l_pupil_pos', ' l_pupil_pos_x', ' l_pupil_pos_y',
                           ' l_frown',' l_squeeze', ' l_wide']]

requestPosX, requestPosY, requestPosZ = makeRequestHmd(pHmd)
requestCombinedX, requestCombinedY, requestCombinedZ = makeRequestEyetracker(pEyetracker)

pBvp, requestBvp = processE4(e4Bvp)
pEda, requestEda = processE4(e4Eda)
pTmp, requestTmp = processE4(e4Tmp)
# pIbi, requestIbi = processE4(e4Ibi)
# pHr, requestHr = processE4(e4Hr)


sendPosX = makeJson(requestPosX)
sendPosY = makeJson(requestPosY)
sendPosZ = makeJson(requestPosZ)
sendCombinedX = makeJson(requestCombinedX)
sendCombinedY = makeJson(requestCombinedY)
sendCombinedZ = makeJson(requestCombinedZ)

sendBvp = makeJson(requestBvp)
sendEda = makeJson(requestEda)
sendTmp = makeJson(requestTmp)
# sendIbi = makeJson(requestIbi)
# sendHr = makeJson(requestHr)


anomalyPosX = getResult(sendPosX)
anomalyPosY = getResult(sendPosY)
anomalyPosZ = getResult(sendPosZ)
anomalyCombinedX = getResult(sendCombinedX)
anomalyCombinedY = getResult(sendCombinedY)
anomalyCombinedZ = getResult(sendCombinedZ)
anomalyBvp = getResult(sendBvp)
anomalyEda = getResult(sendEda)
anomalyTmp = getResult(sendTmp)
# anomalyIbi = getResult(sendIbi)
# anomalyHr = getResult(sendHr)


{'expectedValues': [-0.5612771599999999, -0.5629899909375, -0.5677704087731482, -0.5707497574419301, -0.5719476825052683, -0.5753216364311942, -0.5769781720561943, -0.5763510391673053, -0.5746826700161116, -0.5740730034643875, -0.573299643967033, -0.5744783801208792, -0.5754390333516485, -0.5767633110989013, -0.5774778266989016, -0.5779282710395609, -0.5762569613142862, -0.5741969163692311, -0.5736441979926742, -0.571007876530605, -0.5690833678303399, -0.5708741386431477, -0.5744566402833595, -0.5744330176585699, -0.5765578710180591, -0.5834368535644782, -0.5829289313589804, -0.5773373418616263, -0.5751860864344014, -0.5733191104184627, -0.5683251681107702, -0.5702395230748885, -0.5753346903500202, -0.5787028605698005, -0.582205901310541, -0.5834303864102559, -0.5826493679487179, -0.5824725568376067, -0.5831959045299143, -0.5837570427728089, -0.5844010777183428, -0.5851269779040187, -0.5857916608669818, -0.5860683720977505, -0.5860540612878061, -0.5860224930089395, -0.585914844012074],

{'expectedValues': [-0.3973437249999999, -0.3893511265624999, -0.3880515189429012, -0.38301392196579653, -0.3808757444691889, -0.37478990539511486, -0.3746776973701149, -0.3742991078293742, -0.3790449454686315, -0.38451305087977206, -0.3932855056681318, -0.3990856503758241, -0.4057350684065934, -0.4111780109890111, -0.4149151338813189, -0.4159542623978025, -0.41561673294725293, -0.4139029144307694, -0.41271247569450564, -0.4112562715289883, -0.4109087154016674, -0.4149351411652141, -0.4162432628527609, -0.4166202875622858, -0.4206236330068221, -0.40827714391252423, -0.4079333701194209, -0.41115892771758844, -0.4143589981728987, -0.4187496433542446, -0.4324014995080908, -0.4327465731420432, -0.43304319499389476, -0.4331007344444444, -0.4317704225925926, -0.43084792373219366, -0.4315742583475782, -0.4314206227920227, -0.4316584250997149, -0.4323319792426954, -0.4326960674605601, -0.43289286334915467, -0.43328001297878416, -0.43372547703006603, -0.43362668559462064, -0.43354192404342273, 

{'expectedValues': [0.024710843500000003, -0.14035668581249997, -0.12894721331944445, -0.17160817895510055, -0.2092564626123563, -0.26586681383087485, -0.2722285052058748, -0.3831779019392081, -0.40407460845849297, -0.39607105977944784, -0.35623127354664363, -0.30363388776202827, -0.18793926859279758, -0.1065017880725276, -0.03047364961529677, -0.0743699980988132, -0.08332968341200005, -0.10371514096694501, -0.1915032951801318, -0.20520006843364902, -0.20366812373338378, -0.20956415350826055, -0.1749262741151388, -0.058471316230523485, -0.06657666650500627, -0.0253525442052716, -0.04511699070871984, -0.09345878826612726, -0.13960922143755586, -0.11510480107829668, -0.12792822571019596, -0.09948564431868405, -0.15956613292423966, -0.1716321757289773, -0.19508832491490313, -0.17700157177303222, -0.15609749980380147, -0.10361924758157919, -0.18800589564387452, -0.24049342844697535, -0.22227358838301026, -0.22469999119202894, -0.2584919907475845, -0.264827175309123, -0.21457706040487262, -

In [39]:
requestBvp['value'].mean()

-0.5292052380952386

In [51]:
client = AnomalyDetectorClient(AzureKeyCredential(SUBSCRIPTION_KEY), ANOMALY_DETECTOR_ENDPOINT)

In [61]:
series = []
data_file = pd.read_csv(TIME_SERIES_DATA_PATH, header=None, encoding='utf-8',  date_parser=[0])
for index, row in data_file.iterrows():
    series.append(TimeSeriesPoint(timestamp=row[0], value=row[1]))

In [62]:
request = DetectRequest(series=series, granularity=TimeGranularity.DAILY)

In [63]:
print('Detecting anomalies in the entire time series.')

try:
    response = client.detect_entire_series(request)
except AnomalyDetectorError as e:
    print('Error code: {}'.format(e.error.code), 'Error message: {}'.format(e.error.message))
except Exception as e:
    print(e)

if any(response.is_anomaly):
    print('An anomaly was detected at index:')
    for i, value in enumerate(response.is_anomaly):
        if value:
            print(i)
else:
    print('No anomalies were detected in the time series.')

Detecting anomalies in the entire time series.


TypeError: catching classes that do not inherit from BaseException is not allowed

In [17]:
print('Detecting the anomaly status of the latest data point.')

try:
    response = client.detect_last_point(request)
except AnomalyDetectorError as e:
    print('Error code: {}'.format(e.error.code), 'Error message: {}'.format(e.error.message))
except Exception as e:
    print(e)

if response.is_anomaly:
    print('The latest point is detected as anomaly.')
else:
    print('The latest point is not detected as anomaly.')

Detecting the anomaly status of the latest data point.


TypeError: catching classes that do not inherit from BaseException is not allowed

In [8]:
print('Detecting change points in the entire time series.')

try:
    response = client.detect_change_point(request)
except AnomalyDetectorError as e:
    print('Error code: {}'.format(e.error.code), 'Error message: {}'.format(e.error.message))
except Exception as e:
    print(e)

if any(response.is_change_point):
    print('An change point was detected at index:')
    for i, value in enumerate(response.is_change_point):
        if value:
            print(i)
else:
    print('No change point were detected in the time series.')

Detecting change points in the entire time series.


TypeError: catching classes that do not inherit from BaseException is not allowed